**Task 1: Pre-process the dataset**

All the imports:

In [1]:
import csv
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import datetime 

**Check how many observations per patient there are**
Just to see what amount of data we're dealing with we want to check how many observations per each patient there are in the dataset.

In [28]:
df = pd.read_csv("./data/dataset_mood_smartphone.csv")
#counted = df.groupby(["time"]).size()
df['time'] = pd.to_datetime(df['time'], errors='coerce')
#df['time'].dtype
counted = df.groupby([df['time'].dt.date]).size()
print (len(counted))
counted = counted.to_frame().reset_index()
counted.columns = ['date', 'size']

#print (counted.columns)
##counted.head()
#print (counted)
counted[counted['size']==counted['size'].min()]
#hist = counted.hist(bins=113)
#plt.figure(figsize=(9,7), dpi=100)
#plt.hist(counted)

#plt.show()


113


,date,size
112,2014-06-09,1


Here we're checking for the amount of observations per each vatiable. 

In [3]:
counted = df.groupby(["variable"]).size()
print (len(counted))
print (counted)

19
variable
activity                22965
appCat.builtin          91288
appCat.communication    74276
appCat.entertainment    27125
appCat.finance            939
appCat.game               813
appCat.office            5642
appCat.other             7650
appCat.social           19145
appCat.travel            2846
appCat.unknown            939
appCat.utilities         2487
appCat.weather            255
call                     5239
circumplex.arousal       5643
circumplex.valence       5643
mood                     5641
screen                  96578
sms                      1798
dtype: int64


There are __***27 patients***__, names from 01 to 33 (some numbers are missing)

There are __***19 variables***__



**Making individual files for each patient:**

In [4]:
RawFile = "dataset_mood_smartphone.csv"

with open(RawFile) as infile:
    reader = csv.reader(infile)
    #input_header = reader.next()
    for row in reader:
        patient = row[1]
        CleanedFile = "./data/" + patient +".csv"
        with open(CleanedFile, 'a', newline='') as outfile:
            writer = csv.writer(outfile)
            if row[4] != "NA": #deleting rows with NA as VALUE
                writer.writerow(row[2:5])

Acording to the course desctiption, each variable will be dealt with as shown below to get a day summary for each patient.

activity                Numerical value - use mean

appCat.builtin          Time - add times

appCat.communication    Time - add times

appCat.entertainment    Time - add times

appCat.finance          Time - add times

appCat.game             Time - add times

appCat.office           Time - add times

appCat.other            Time - add times

appCat.social           Time - add times

appCat.travel           Time - add times

appCat.unknown          Time - add times

appCat.utilities        Time - add times

appCat.weather          Time - add times

call                    Numerical value - add

circumplex.arousal      Numerical value - use mean

circumplex.valence      Numerical value - use mean

mood                    Numerical value - use mean

screen                  Time

SMS                     Numerical values - add



In [2]:
df = pd.read_csv("./data/AS14.05.csv")
counted = df.groupby(["variable"]).size()
print (len(counted))
print (counted)


FileNotFoundError: File b'./data/AS14.05.csv' does not exist

In [6]:
df.dtypes

time         object
variable     object
value       float64
dtype: object




**Want to creat a table/csv that would summarize each day. Then we can proceed with another algorithm.**

Produce a day by day summary of the data
---

In [7]:
variables= ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office','appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
patient = ['01', '02','03','05','06','07','08','09','12','13','14','15','16','17','19','20','23','24','25','26','27','28','29','30','31','32','33']
for patientnumber in patient:
    inpath = "./data/AS14."+patientnumber+".csv"
    outpath = "./data/summary/AS14." + patientnumber + ".csv"
    df = pd.read_csv(inpath)

    df["time"] = pd.to_datetime(df["time"]) 
    #df['date_minus_time'] = df["time"].apply( lambda df : datetime.datetime(year=df.year, month=df.month, day=df.day))
    #df.set_index(df["date_minus_time"],inplace=True) # using date as index 
    df.index = df['time']
    df = df.sort_index()


    byday = [] #variables summmed/meaned for each day
    for i, j in enumerate(variables):
        #print(i, j)
        dfvariables = df.loc[df['variable'] == j]
        if i < 4:
            data = dfvariables['value'].resample('D').mean()
    #         print(i, j, len(data))
    #         print (len(data))
        else:
            data = dfvariables['value'].resample('D').sum()
            #print(i, j, len(data))
            #print (len(data))
        byday.append(data)

    daysSet = pd.concat(byday, axis=1)
    daysSet.columns = variables
    daysSet= daysSet.replace(0, np.nan)

    #print(daysSet)
    daysSet.to_csv(outpath, encoding='utf-8', index=True)
    

In [9]:
# variables= ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office','appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
# patient = ['01', '02','03','05','06','07','08','09','12','13','14','15','16','17','19','20','23','24','25','26','27','28','29','30','31','32','33']

# for patientnumber in patient:
#     inpath = "./data/summary/AS14." + patientnumber + ".csv"
#     outstats = "./data/stats/" + patientnumber + ".txt"
   
#     df = pd.read_csv(inpath)
#     with open(outstats, 'a', newline='') as statsFile:
        
#         for i in variables:
#             j = df[i]
            
#             nanSum = j.isna().sum()
#             legitVal = len(j) - nanSum
#             statsFile.write("There are " + str(legitVal) + " instances of " + i + "\n")
#             #array.append(legitVal)
#             #statsFile.write(". For each day it is \n")
#             #statsFile.write(str(j) + "\n")
#             del j

        
        

We noticed that some of the data variables only appear for few days so we want to check the frequency of each variable of each patient to maybe be able to delete some columns (variables) with very few values.

So here we're are creating the frequency summary for each patient for each variable. 

In [5]:
variables= ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office','appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
patient = ['01', '02','03','05','06','07','08','09','12','13','14','15','16','17','19','20','23','24','25','26','27','28','29','30','31','32','33']
matrixForAll = ['patientNumber', 'mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office','appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
outstats = "./data/stats/AllStatsFrequency.csv"
with open (outstats,'a', newline='') as statsFile:

    writer = csv.writer(statsFile)
    writer.writerow(matrixForAll)

    for patientnumber in patient:
        inpath = "./data/summary/" + patientnumber + "_compressed.csv"
        df = pd.read_csv(inpath)
        array = []
        array.append(patientnumber)

        for i in variables:
            j = df[i]

            nanSum = j.isna().sum()
            legitVal = len(j) - nanSum
            array.append(legitVal)
            del j
            
        writer.writerow(array)

We also want to explore some statistics aboout the data so we create a sum/mean values that correspont to each patient for each variable and put it in a csv for further analysis.  

In [1]:
variables= ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office','appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
patient = ['01', '02','03','05','06','07','08','09','12','13','14','15','16','17','19','20','23','24','25','26','27','28','29','30','31','32','33']
matrixForAll = ['patientNumber', 'mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office','appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
outstats = "./data/stats/AllStats.csv"
with open (outstats,'a', newline='') as statsFile:

    writer = csv.writer(statsFile)
    writer.writerow(matrixForAll)

    for patientnumber in patient:
        inpath = "./data/summary/AS14." + patientnumber + ".csv"
        df = pd.read_csv(inpath)
        array = []
        array.append(patientnumber)

        for k, i in enumerate(variables):
            j = df[i]
            mean = j.mean()
            if k < 4:
                data = j.mean()
    #         print(i, j, len(data))
    #         print (len(data))
            else:
                data = j.sum()
            array.append(data)
            del j
            
        writer.writerow(array)
    #print(matrixForAll)

NameError: name 'csv' is not defined

In [3]:

# Normalize the data attributes for the Iris dataset.
#from sklearn.datasets import load_iris
from sklearn import preprocessing
patient = ['01', '02','03','05','06','07','08','09','12','13','14','15','16','17','19','20','23','24','25','26','27','28','29','30','31','32','33']
for patientnumber in patient:
    inpath = "./data/individualPatientData/AS14."+patientnumber+".csv"
    df = pd.read_csv(inpath)
    # load the iris dataset
    iris = df.load_iris()
    print(iris.data.shape)
    # separate the data from the target attributes
    X = iris.data
    y = iris.target
    # normalize the data attributes
    normalized_X = preprocessing.normalize(X)



AttributeError: 'DataFrame' object has no attribute 'load_iris'

Next up we try to transform the data in such way that it can be all be valuable. 

## Delete Unnecessary Rows
## Perform One-Hot-Encoding for day of week (day_0 = monday, and so on...)

In [11]:
variables= ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office','appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
patient = ['01', '02','03','05','06','07','08','09','12','13','14','15','16','17','19','20','23','24','25','26','27','28','29','30','31','32','33']
for patientnumber in patient:
    inpath = "./data/summary/AS14." + patientnumber + ".csv"
    outpath = "./data/summary/" + patientnumber + "_compressed.csv"
    df = pd.read_csv(inpath)

    #df.dtypes
    #print (df) #cheched datatype of each column

    #df = df.dropna(subset=["mood"], inplace=True) #drops the rows with empty mood

    df.dropna(thresh=4, inplace=True)     # drop the rows with at least 4 LEGIT values 
                                                # GOOD TO START WITH CAUSE THERE IS NO POINT IN FILLING MOOD DATA 
                                                # WHEN THERE IS NO OTHER DATA IN THAT ROW
                                                # MIGHT NEED TO CHANGE TO 3

    # Most of this is 0 values: 
    # game
    # office
    # weather 
    # Probably more accurate to fill it in with 0

    # Rest - median 

    # s=df.isnull().cumsum()
    # t1=df[(s==1).shift(-1).fillna(False)].stack().reset_index(level=0,drop=True)
    # t2=df.lookup(s.idxmax()+1,s.idxmax().index)
    # df.fillna(t1/(2**s)+t2*(1-0.5**s)*2/2)

    #df.mood.fillna(method='bfill',inplace=True)  #for back fill - THIS IS CURRENTLY DONE IN THE DATA - This is good for MOOD
    #df.mood.fillna(method='ffill',inplace=True)  #for forward-fill - THIS IS CURRENTLY DONE IN THE DATA - This is good for MOOD

    #df.fillna({'game':0, 'office':0, 'weather': 0}, inplace=True) #fill in the empty rows "call" and "sms" with 0


    #df.fillna(df.mood.mean(),inplace=True)       # use mean of the data to fill missind data - this not not good for MOOD (too much difference)

    #df.mood.fillna(df.mood.median(),inplace=True)     # use median to till in missing data - this not not good for MOOD

    #df = df.fillna(0) #fill in the empty rows with 0
    #df.fillna({'game':0, 'office':0, 'weather': 0}, inplace=True) #fill in the empty rows "call" and "sms" with 0


    #print (df)
    
    df['time'] = pd.to_datetime(df['time'])
    df['dayofweek'] = df['time'].dt.dayofweek
    one_hot = pd.get_dummies(df['dayofweek'], prefix = 'day')
    df = df.drop('dayofweek', axis=1)
    df = df.join(one_hot)

    df.to_csv(outpath, encoding='utf-8', index=False)
    



Data standardization 
---

In [43]:
inpath = "./data/table_5.csv"
outpath = "./data/table_5_std.csv"
variables= ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office','appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
df = pd.read_csv(inpath)
df.describe()
for variable in variables:
    #print (df[variable].mean())
    if variable in df.columns:
        meanVal = df[variable].mean()
        stdVal = df[variable].std()
        df[variable] = (df[variable]-meanVal)/stdVal
        #df.apply([variable]-[variable].mean())
        #df.apply(lambda x: x-x.mean())
df.describe()

df.to_csv(outpath, encoding='utf-8', index=False)